In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Setup Selenium ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Base URL without the page index
base_url = "https://www.lcwaikiki.iq/en-US/IQ/product-group/women?PageIndex={}"

# List to store all scraped data
all_products_data = []

# Loop through the pages
for page_index in range(1, 203):  # 202 pages in total
    # Navigate to the current page
    driver.get(base_url.format(page_index))
    time.sleep(5)  # Allow time for the page to load. Consider using a more reliable wait condition here.

    # Scrape the product data with the current filter applied
    products_data = []
    product_elements = driver.find_elements(By.CSS_SELECTOR, '.product-card')
    for product in product_elements:
        # Scroll each product into view to load its image
        driver.execute_script("arguments[0].scrollIntoView(true);", product)
        time.sleep(1)  # Allow time for the image to load
        
        product_name = product.find_element(By.CSS_SELECTOR, '.product-card__title').text
        original_price = product.find_element(By.CSS_SELECTOR, '.product-price__price').text
        price_in_cart_elements = product.find_elements(By.CSS_SELECTOR, '.product-price__cart-price-badge-price')
        price_in_cart = price_in_cart_elements[0].text if price_in_cart_elements else 'Not Available'
        image_url_elements = product.find_elements(By.CSS_SELECTOR, '.product-image__image[src]')
        image_url = image_url_elements[0].get_attribute('src') if image_url_elements else 'Not Available'
        
        products_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Price In Cart': price_in_cart,
            'Image URL': image_url
        })
    
    all_products_data.extend(products_data)

# Convert the list of data into a pandas DataFrame
df = pd.DataFrame(all_products_data)

# Save the DataFrame to an Excel file
df.to_excel('products_data.xlsx', index=False)

# Close the browser
driver.quit()
